In [32]:
import pandas as pd
import numpy as np

In [33]:
context_df = pd.read_csv('term_context.csv', sep='\t').dropna()

In [34]:
context_without_term = []

for term, context in context_df[['term', 'context']].iloc:
    context_without_term.append(context.replace(term, ' [TERM] '))
    
context_df['context_without_term'] = context_without_term

In [35]:
from normalizer import normalize_sentences

In [36]:
abbrs = set(context_df['abbreviation'].str.lower())

sentences = normalize_sentences(
    context_df['context_without_term'], 
    except_words=abbrs,
    as_sentences=False
)
clean_sentences = [" ".join([" ".join(word) for word in text]) for text in sentences]
context_df['normalized_context'] = clean_sentences

Cleaning sentences:   0%|          | 0/10295 [00:00<?, ?it/s]

Normalizing words:   0%|          | 0/7039 [00:00<?, ?it/s]

In [167]:
context_df[context_df.abbreviation == 'Hb'].term.value_counts()

гемоглобин    1466
Name: term, dtype: int64

In [174]:
context_df.groupby('abbreviation').term.unique().str.len().sort_values().tail(60)

abbreviation
ММ      1
МО      1
МП      1
МПВ     1
МПЗ     1
МПЭ     1
МР      1
МРТ     1
МТС     1
КК      1
КИ      1
КШ      1
КВ      1
ЖНР     1
ЖТ      1
ЗГТ     1
ИА      1
ИБС     1
ИГ      1
ИГХ     1
ИИ      1
ИКД     1
ИЛАГ    1
КГ      1
ИМ      1
ИМТ     1
ИП      1
ИСТ     1
уЛП     1
ИЦН     1
КАГ     1
ИФР     1
ИЭ      1
ИФТ     1
МТ      2
СТ      2
ЖЭ      2
в/в     2
НЯ      2
ОА      2
ГД      2
ЗНО     2
КС      2
ГТ      2
БА      2
ЛУ      2
АГ      2
АД      2
ПТГ     2
ПТ      2
ТГ      2
РЧА     2
ПЖ      2
ПЗ      2
ИТ      2
ПР      2
ФЭК     2
ДН      3
КА      3
ЛП      3
Name: term, dtype: int64

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [49]:
vectorizer = TfidfVectorizer()
char_vectorizer = CountVectorizer(analyzer='char')

points = vectorizer.fit_transform(context_df['normalized_context']).toarray()
char_points = char_vectorizer.fit_transform(context_df['abbreviation']).toarray()

In [84]:
import joblib

joblib.dump(vectorizer, 'context_vectorizer.joblib')
joblib.dump(char_vectorizer, 'char_vectorizer.joblib')

['char_vectorizer.joblib']

In [39]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [176]:
df_train, df_test, X_train, X_test, y_train, y_test = train_test_split(
    context_df,
    np.hstack((points, char_points)),
    context_df.term, 
    test_size=0.3
)

In [61]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train) 

RandomForestClassifier()

In [85]:
joblib.dump(clf, 'decoder_model.joblib')

['decoder_model.joblib']

In [62]:
print('train accuracy:', clf.score(X_train, y_train))
print('train f1_score:', f1_score(y_train, clf.predict(X_train), average='weighted'))

print('test accuracy:', clf.score(X_test, y_test))
print('test f1_score:', f1_score(y_test, clf.predict(X_test), average='weighted'))

train accuracy: 1.0
train f1_score: 1.0
test accuracy: 0.9514405956620265
test f1_score: 0.9374111313268304


In [180]:
df_test = df_test.copy()
df_test['predicted_term'] = clf.predict(X_test)
df_test['proba']= np.max(clf.predict_proba(X_test), axis=1)

df_test[df_test.predicted_term != df_test.term]

,event_code,mcb_code,abbreviation,term,context,context_without_term,normalized_context,predicted_term,proba
2127,GACAc+РAAAABl090009#AD,I10,СКФ,скорость клубочковой фильтрации,"глюкоза-10.75, алт-60, аст-145, креат-116. ско...","глюкоза-10.75, алт-60, аст-145, креат-116. [T...",глюкоза алта аста креат term по формула ckd epi,холестерин,0.24
7568,GACAVпKABrAAB12:41A$AD,I10,НУЖТ,неустойчивая желудочковая тахикардия,на хм-экг регистрировалась пароксизмальная неу...,на хм-экг регистрировалась пароксизмальная [T...,на хм экг регистрироваться пароксизмальный ter...,желудочковая тахикардия,0.27
7807,GACAR+OAAIAAJ10:00-eAD,I10,ПЭ,преэклампсия,23-24 недели находилась с 18-25.08.15 преэклам...,23-24 недели находилась с 18-25.08.15 [TERM] ...,неделя находиться с term лёгкий степень в ж к ...,эндопротезирование,0.20
2023,GACALфМAAGAAg120012#AB,I10,ГВК,гомованилиновая кислота,ознакомлена. ванилииминдалевая кислота -2 гомо...,ознакомлена. ванилииминдалевая кислота -2 [TE...,ознакомить ванилииминдалевый кислота term гидр...,коронарография,0.52
8047,GACA]2&AAAAKW141014#AD,I10,ОПТГ,ортопантомограмма,очередной курс терапии начат 20.05.2018: ортоп...,очередной курс терапии начат 20.05.2018: [TER...,очередной курс терапия начать term от патологи...,паратгормон,0.24
8061,GACAJ=ЙAVfAAANABАпB$AB,I10,ФН,фебрильная нейтропения,"степени, нейтропения 3-4 степени. фебрильная н...","степени, нейтропения 3-4 степени. [TERM] 3ст...",степень нейтропение степень term ст с разрешен...,фибрилляция предсердий,0.11
7366,GACAVlсAAiAAA09:47A#AD,I10,КАГ,коронароангиография,"стационарах города, в 2015г выполнялась корона...","стационарах города, в 2015г выполнялась [TERM...",стационар город в г выполняться term в нмица а...,ангиография,0.37
10037,GACA]4^AABAAB10:40-eAD,I10,ГА,гонартроз,"делали денситометрию - остеопороз, есть гонарт...","делали денситометрию - остеопороз, есть [TERM...",делать денситометрия остеопороз есть term стад...,ангиография,0.42
1845,GACAIxoAADAAA21:34B%AD,I10,ПЗ,прогрессия заболевания,"патогенетической, поддерживающей терапии, посл...","патогенетической, поддерживающей терапии, посл...",патогенетический поддерживать терапия последни...,поле зрения,0.15
233,GACAgА9ABBAAC11:40A$AD,I10,ВГЛУ,внутригрудные лимфатические узлы,и сегментарные бронхи проходимы&#59;- внутригр...,и сегментарные бронхи проходимы&#59;- [TERM] ...,и сегментарный бронх проходимый term не увелич...,лимфатические узлы,0.44


- Улучшить задачку какую нибудь

In [159]:
class AbbreviationDecoder:
    def __init__(self):
        self.context_vectorizer = joblib.load('context_vectorizer.joblib')
        self.char_vectorizer = joblib.load('char_vectorizer.joblib')
        self.model = joblib.load('decoder_model.joblib')
    
    def _normalize_sentences(self, sentences):
        sentences = normalize_sentences(
            sentences, 
            except_words=abbrs,
        )
        clean_sentences = [" ".join([" ".join(word) for word in text]) for text in sentences]
        
        return clean_sentences
    
    def _get_features(self, context, abbreviation):
        if isinstance(context, str):
            context = [context]
            abbreviation = [abbreviation]
            
        context = self._normalize_sentences(context)
            
        context_features = vectorizer.transform(context).toarray()
        abbr_features = char_vectorizer.transform(abbreviation).toarray()

        features = np.hstack((context_features, abbr_features))

        return features

    def predict(self, abbreviation, context):
        features = self._get_features(context, abbreviation)
        return self.model.predict(features)

### Evaulation

In [160]:
records_df = pd.read_csv('../data/I10_anamnesis.csv', sep='\t',)

In [161]:
records_df[records_df.RecordEMC.str.findall(' КТ ').str.len() != 0].RecordEMC.iloc[0]

"<w id='GACAh;' > <v id='AA~1;' > ОНМК отрицает. Считает себя больной с 2012 года, когда стали беспокоить вышеуказанные симптомы. Стентирование левой ВСА от 2012 года. По данным КТ от 2020 года: стеноз ПВСА 90%. Настоящая госпитализация для стентирования ПВСА </v> </w>"

In [162]:
decoder = AbbreviationDecoder()

In [190]:
decoder.predict('ЛП', 'воспалительный изменение')

Cleaning sentences:   0%|          | 0/1 [00:00<?, ?it/s]

Normalizing words:   0%|          | 0/2 [00:00<?, ?it/s]

array(['люмбальная пункция'], dtype=object)

может ли являтся аббревиатура настоящей аббревиатурой для слова